In [2]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [20]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/andreasantoro/split-garbage-dataset')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 81.7M/81.7M [00:00<00:00, 92.8MB/s]


In [21]:
train ='/content/split-garbage-dataset/train'
validation_dir = '/content/split-garbage-dataset/valid'

In [22]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [23]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

6 Classes
1768 Train images
328 Test images


In [15]:
os.rmdir('/content/waste-segregation-image-dataset/Dataset/val/.ipynb_checkpoints')

In [24]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)
img_width,img_height =128,128
input_shape=(img_width,img_height,3)
batch_size =32
train_generator =train_datagen.flow_from_directory(train,
                                target_size=(img_width,img_height),batch_size=batch_size)
test_generator=test_datagen.flow_from_directory(validation_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 1768 images belonging to 6 classes.
Found 328 images belonging to 6 classes.


In [12]:
od.download('https://www.kaggle.com/datasets/keras/inceptionresnetv2')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 392M/392M [00:05<00:00, 80.8MB/s]


In [28]:
from keras import Model
from keras import optimizers
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dense

model_dir = "../content/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
model = Sequential()
model.add( tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights= model_dir, input_shape=(128, 128, 3)))

model.add(GlobalAveragePooling2D())
model.add(Dense(6, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_2   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 6)                 9222      
                                                                 
Total params: 54,345,958
Trainable params: 54,285,414
Non-trainable params: 60,544
_________________________________________________________________


In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(train_generator,validation_data=test_generator,epochs=50,shuffle=True,callbacks=early_stopping_callback)

Epoch 1/50
56/56 [==============================] - 14s 253ms/step - loss: 0.5503 - accuracy: 0.8264 - val_loss: 30.1227 - val_accuracy: 0.4238
Epoch 2/50
56/56 [==============================] - 14s 240ms/step - loss: 0.4690 - accuracy: 0.8569 - val_loss: 28.5714 - val_accuracy: 0.6768
Epoch 3/50
56/56 [==============================] - 13s 238ms/step - loss: 0.4033 - accuracy: 0.8812 - val_loss: 6.3459 - val_accuracy: 0.2409
Epoch 4/50
56/56 [==============================] - 14s 244ms/step - loss: 0.5518 - accuracy: 0.8224 - val_loss: 5.4951 - val_accuracy: 0.6128
Epoch 5/50
56/56 [==============================] - 13s 234ms/step - loss: 0.4182 - accuracy: 0.8626 - val_loss: 1.7384 - val_accuracy: 0.6159
Epoch 6/50
56/56 [==============================] - 13s 235ms/step - loss: 0.3473 - accuracy: 0.8908 - val_loss: 0.9235 - val_accuracy: 0.6951
Epoch 7/50
56/56 [==============================] - 13s 232ms/step - loss: 0.2681 - accuracy: 0.9191 - val_loss: 2.4916 - val_accuracy: 0.58

In [32]:
model.save('inceptionresnetv2_model.h5')

In [34]:
model.evaluate(test_generator,verbose=1)

11/11 [==============================] - 1s 92ms/step - loss: 0.4988 - accuracy: 0.8780


[0.49878793954849243, 0.8780487775802612]